In [1]:
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

In [4]:


exmaple_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=exmaple_prompt,
    examples= examples,
)
print(few_shot_prompt.format())

Human: 2+2
AI: 4
Human: 2+3
AI: 5


In [13]:
from langchain_openai import ChatOpenAI
import os
from openai import api_key, base_url
from transformers import Qwen2_5_VLConfig

from langchain_core.output_parsers.string import StrOutputParser
from langchain.schema import output_parser
# llm = ChatOpenAI(model=)
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "sk-C3gsaWJaOsmjrNv1fIobc4zEEsEkD15T98j80Vc8mNuSE420")
# llm = ChatOpenAI(model="o3-mini", api_key=OPENAI_API_KEY, base_url="https://api.deepbricks.ai/v1/")
QWEN_API_KEY = os.getenv("QWEN_API_KEY", "sk-ypjnechwutigvbyglbkguukzsmzkkxfibauydwkbjrypwojd")
model = ChatOpenAI(model="Qwen/Qwen3-30B-A3B-Instruct-2507", api_key=QWEN_API_KEY, base_url="https://api.siliconflow.cn/v1")



output_parser = StrOutputParser()

In [14]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个非常聪明的数学天才"),
        few_shot_prompt,
        ("human", "{input}")
    ]
)


chain = final_prompt | model | output_parser


chain.invoke({"input": "2+5"})

'7'

In [15]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector, SemanticSimilarityExampleSelector
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embedding_path = "/home/zpc/langchain/blibli/bge-large-zh-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=embedding_path)




/tmp/ipykernel_129003/1124703204.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_path)


In [16]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "牛对月亮说了什么？", "output": "什么都没有"},
    {
        "input": "给我写一首关于月亮的五言诗",
        "output": "月儿挂枝头，清辉洒人间。 银盘如明镜，照亮夜归人。 思绪随风舞，共赏中秋圆。",
    },
]

to_vectorize = [" ".join(example.values()) for example in examples]
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [17]:
    example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "对牛弹琴"})

[{'input': '牛对月亮说了什么？', 'output': '什么都没有'}, {'input': '2+2', 'output': '4'}]

In [18]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

In [19]:
few_shot_prompt.format(input="What's 3+3?")

'Human: 2+3\nAI: 5\nHuman: 2+4\nAI: 6'